# DESARROLLO DE VISUALIZACIÓN - PEC 3

## Integrantes: Alejandro Sanz García y Rubén Cumbreño Juan


## Link del dataset

https://www.kaggle.com/zynicide/wine-reviews#winemag-data_first150k.csv

### Requisitos

Parte de análisis:
- Jupyter Notebook
- Python 3.X

Parte web:
- En principio deberia funcionar tanto en python 2.X como en 3.X (testeado en ambos)
- Testeado en Chrome
- Necesario tener instalado django 1.11 o superior (testeado con la 1.11.13)

## Limpieza y Análisis de datos

### Cabeceras

- ''  '': El id del campo
- country: el pais del vino
- description: descripcion del vino
- designacion: denominación de origen
- points: puntos del vino
- price: precio del vino
- province: provincia del pais
- region_1: primera region perteniciente al pais de ese vino
- region_2: segunda region perteniciente al pais de ese vino
- variety: tipo de vino
- winery: viñedo al que pertenece

Lo primero es ver la cantidad de datos que tenemos en el dataset y como se encuentran:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt


df= pd.read_csv("winemag-data_first150k.csv")

vinos_pais = df[["country"]].groupby("country").size()

print(vinos_pais)

country
Albania                       2
Argentina                  5631
Australia                  4957
Austria                    3057
Bosnia and Herzegovina        4
Brazil                       25
Bulgaria                     77
Canada                      196
Chile                      5816
China                         3
Croatia                      89
Cyprus                       31
Czech Republic                6
Egypt                         3
England                       9
France                    21098
Georgia                      43
Germany                    2452
Greece                      884
Hungary                     231
India                         8
Israel                      630
Italy                     23478
Japan                         2
Lebanon                      37
Lithuania                     8
Luxembourg                    9
Macedonia                    16
Mexico                       63
Moldova                      71
Montenegro                    2


Observamos como muchos paises continen pocos vinos en esta base de datos, lo que provocaría la obtención de resultados engañosos. Por lo que decidimos quedarnos solo con los vinos de los paises con mas de 500 vinos

In [2]:
paises_muchos = vinos_pais[vinos_pais>500]
paises = paises_muchos.index.values.tolist()
df_wine = df[df.country.isin(paises)]
vinos_paisf = df_wine[["country"]].groupby("country").size()

print("Paises finalmente incluidos en el analisis / vinos por pais")
print(vinos_paisf)

#Generamos csv para nuestra aplicacion
vinos_paisf.to_csv('bbdd/vinos_por_pais.csv', encoding="utf-8")

Paises finalmente incluidos en el analisis / vinos por pais
country
Argentina        5631
Australia        4957
Austria          3057
Chile            5816
France          21098
Germany          2452
Greece            884
Israel            630
Italy           23478
New Zealand      3320
Portugal         5322
South Africa     2258
Spain            8268
US              62397
dtype: int64


Una vez filtrados los paises, procedemos al analisis de los datos

##### Tipo de dato de nuestro dataset

In [3]:
df_wine.dtypes

Unnamed: 0       int64
country         object
description     object
designation     object
points           int64
price          float64
province        object
region_1        object
region_2        object
variety         object
winery          object
dtype: object

##### Vemos valores nulos de nuestro dataset

In [4]:
df_wine.isnull().sum()

Unnamed: 0         0
country            0
description        0
designation    45392
points             0
price          13658
province           0
region_1       23894
region_2       88615
variety            0
winery             0
dtype: int64

##### Vemos un resumen estadistico de los datos numéricos

In [5]:
df_wine.describe()

,Unnamed: 0,points,price
count,149568.000000,149568.000000,135910.000000
mean,75510.535262,87.902118,33.190619
std,43547.364794,3.220441,36.343701
min,0.000000,80.000000,4.000000
25%,37843.750000,86.000000,16.000000
50%,75507.500000,88.000000,24.000000
75%,113250.250000,90.000000,40.000000
max,150929.000000,100.000000,2300.000000


##### Precio medio por pais:

In [6]:
grouped = df_wine[["country","price"]].groupby("country")

country_price_avg = grouped.mean().sort_values(by="price", ascending=False)

print("Precios medios por País")
print(country_price_avg)

#Generamos csv para nuestra aplicacion
country_price_avg.to_csv('bbdd/precio_medio.csv', encoding="utf-8")

Precios medios por País
                  price
country                
France        45.619885
Germany       39.011078
Italy         37.547913
US            33.653808
Israel        31.304918
Australia     31.258480
Austria       31.192106
Spain         27.048529
Portugal      26.332615
New Zealand   24.173290
Greece        21.747706
South Africa  21.130532
Argentina     20.794881
Chile         19.344780


##### Puntos medios por pais

In [7]:
grouped_points= df_wine[["country","points"]].groupby("country")
country_points_avg = grouped_points.mean().sort_values("points", ascending=False)

print("Puntos medios por País")
print(country_points_avg)

#Generamos csv para nuestra aplicacion
country_points_avg.to_csv('bbdd/calidad_media.csv', encoding="utf-8")

Puntos medios por País
                 points
country                
Austria       89.276742
France        88.925870
Germany       88.626427
Italy         88.413664
Portugal      88.057685
Australia     87.892475
US            87.818789
New Zealand   87.554217
South Africa  87.225421
Israel        87.176190
Spain         86.646589
Chile         86.296768
Greece        86.117647
Argentina     85.996093


##### Precio medio por puntuacion

In [8]:
grouped_price_rate= df_wine[["price","points"]].groupby("points")

price_rate= grouped_price_rate.mean().sort_values(by="price",ascending=False)
print("")
print("Precio pedio por puntuación")
print(price_rate)

#Generamos csv para nuestra aplicacion
price_rate.to_csv('bbdd/precio_medio_por_puntuacion.csv', encoding="utf-8")


Precio pedio por puntuación
             price
points            
100     401.583333
99      289.525000
98      232.131579
97      178.000000
96      132.376991
95      108.630034
94       81.784451
93       66.446115
92       52.296379
91       43.696013
90       37.390133
89       32.524210
88       27.820850
87       24.617444
86       21.815653
85       20.038074
84       19.394845
82       19.191667
83       18.030959
81       17.639551
80       17.340672


##### Relacion calidad/precio por pais

In [9]:
calidad_precio= df_wine["points"]/ (df_wine["price"]* 100/max(df_wine["price"]))
df_wine["calidad_precio"] = calidad_precio
grouped_calidad_precio = df_wine[["country","calidad_precio"]].groupby("country")
quality_price = grouped_calidad_precio.mean().sort_values("calidad_precio", ascending=False)

print("Relación calidad-precio por País")
print(quality_price)

Relación calidad-precio por País
              calidad_precio
country                     
Chile             139.497608
Argentina         135.856721
Portugal          128.706695
South Africa      124.502617
Spain             121.360848
Greece            112.211705
Australia         104.200522
New Zealand       100.776237
Germany            88.354020
Austria            86.540165
France             86.156081
Italy              85.730265
Israel             85.345127
US                 83.177723


C:\Users\JIMMY\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Vamos a obtener los datos referentes a "Israel" usados para la web:

In [10]:
#Todos los datos pertenecientes a Israel
datos_israel = df_wine.loc[df['country'] == 'Israel']
datos_israel['description'] = datos_israel['description'].str.replace('\r\n',' ')

#Generamos csv para nuestra aplicacion (con separador '€' ya que no existe en nuestro conjunto y nos sirve para facilitar el proceso)
datos_israel.to_csv('bbdd/data_israel.csv', sep='€', encoding="utf-8")

C:\Users\JIMMY\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
#Provincias pertenecientes a Israel
array_provincias_israel = datos_israel['province'].unique()
provincias_israel = pd.DataFrame(array_provincias_israel)

#Generamos csv para nuestra aplicacion
provincias_israel.to_csv('bbdd/provincias_israel.csv', sep=';', header=False, encoding="utf-8")

Por último, realizaremos el coeficiente de Pearson para ver la correlación entre el precio y la puntuación

In [12]:
from scipy import stats


Pearson= stats.pearsonr(df_wine.dropna().price.tolist(), df_wine.dropna().points.tolist())
print("Coeficiente Pearson, P-valor")
print(Pearson)

Coeficiente Pearson, P-valor
(0.4356277058255496, 0.0)


Con el analisis del coeficiente de pearson obtenemos que existe un correlación positiva entre el precio y la puntucíon que obtienen los vinos, ademas podemos afirmar que esta correlación es estadisticamente significativa ya que el P-valor es 0

### Los csv generados son los que alimentarán nuestra aplicación como si fueran documentos o tablas de una bbdd